In [19]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
from tqdm import tqdm
importlib.reload(ass)
import ast

In [2]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [3]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\proAnubis_240403_0227.h5')

In [3]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\threeHourRun_24_3_1.h5')
# thisData = [sublist[:94230] for sublist in thisData]

In [4]:
# thisData = [sublist[94230:188460] for sublist in thisData]

In [4]:
etaHits, phiHits = ass.divideHitCountsByRPC_Timed(thisData)

In [5]:
df = ass.remake_data(thisData, mapping)

In [6]:
df_clean = df[df['time'] <= 500]
df_clean = df_clean[df_clean['time'] >= 0]

In [7]:
cluster_df = ass.calculate_cluster_metrics_better(df_clean, 15)

In [8]:
remove_noisy_strips_with_map = partial(ass.remove_noisy_strips, noisy_strips=strips_on_steroid)
anti_steriod_df = cluster_df[cluster_df.apply(lambda row: remove_noisy_strips_with_map(row), axis=1)]

In [9]:
def filter_groups(group):
    has_eta = 'eta' in group['strip_direction'].values
    has_phi = 'phi' in group['strip_direction'].values
    return has_eta and has_phi

filtered_cluster = anti_steriod_df.groupby(['event_number', 'rpc_number']).filter(filter_groups)

In [10]:
fully_massaged_df = filtered_cluster[filtered_cluster['size'] <= 3]

In [11]:
strip_locations = np.array(fully_massaged_df.iloc[0]['locations'])
non_zero_locations = strip_locations[strip_locations != 0]
if non_zero_locations.size > 0:
    strip_location = np.mean(non_zero_locations)

In [13]:
scale_eta = 2.5
scale_phi = 2.5
var_eta = (scale_eta * scale_eta) / 12  
var_phi = (scale_phi * scale_phi) / 12 
def calculate_metric_for_combo(combo, rpc_heights, direction):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])
    weights = 1 /  (uncertainties ** 2)
    try:
        coeffs, res, _, _, _ = np.polyfit(heights, locations, 1, full=True)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf] 
    # slope, intercept = 1/coeffs[0], -coeffs[1]/coeffs[0]
    slope, intercept = coeffs
    predictions = slope * heights + intercept
    residuals = locations - predictions
    weighted_residuals = residuals ** 2
    RSS = np.sum(weighted_residuals)
    sigma_squared = var_eta if direction == 'eta' else var_phi
    chi_squared = np.sum(res / sigma_squared)
    chi_squared_norm = chi_squared / (len(combo)-2) 
    # chi_squared_norm = chi_squared

    return slope, intercept, chi_squared_norm, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (0, 15),
    ('rpc0', 'phi'): (0, 15),
    ('rpc1', 'eta'): (0, 15),
    ('rpc1', 'phi'): (0, 15),
    ('rpc2', 'eta'): (0, 15),
    ('rpc2', 'phi'): (0, 15),
    ('rpc3', 'eta'): (0, 15),
    ('rpc3', 'phi'): (0, 15),
    ('rpc4', 'eta'): (0, 15),
    ('rpc4', 'phi'): (0, 15),
    ('rpc5', 'eta'): (0, 15),
    ('rpc5', 'phi'): (0, 15),
}
    paths = []

    adjusted_muon_speed_cm_ns = 20

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.8, 
        'rpc2': 1.6, 
        'rpc3': 61.6, 
        'rpc4': 120.8, 
        'rpc5': 121.6
    }


    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        if len(event_group) > 50:
            continue
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = scale_eta if direction == 'eta' else scale_phi
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = np.mean(non_zero_locations)
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.min(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    event_time_diff = (np.max(row['times']) - np.min(row['times'])) / len(row['times'])
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling)
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'event_time_diff': event_time_diff,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            for combo_size in range(n, len(unique_rpcs) + 1):
                for combo in combinations(all_clusters, combo_size):
                    if len({c['rpc'] for c in combo}) != len(combo):
                        continue  # Skip combinations where RPCs are not unique

                    time_diffs_are_valid = True
                    for i in range(len(combo)):
                        for j in range(i + 1, len(combo)):
                            error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                            height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                            time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                            expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                            uncertainty_margin = 0

                            if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                                time_diffs_are_valid = False
                                break
                        if not time_diffs_are_valid:
                            break

                    if not time_diffs_are_valid:
                        continue  # Skip to the next combination if time differences are invalid

                    # Calculate metrics only for valid combinations
                    metric = calculate_metric_for_combo(combo, rpc_heights, direction)
                    if metric[2] <= 3.5:
                        valid_combinations.append(metric)
                        

                for valid_combination in valid_combinations:
                    paths.append({
                        'Event Number': event_number,
                        'Direction': direction,
                        'Slope': valid_combination[0],
                        'Intercept': valid_combination[1],
                        'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                        'chi2 norm': valid_combination[2],
                        'RSS': valid_combination[3]
                    })

    path_df = pd.DataFrame(paths)
    return path_df

Chi2 Analysis Best here


In [20]:
Reconstructed_df_6 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 3)

Processing Events:   0%|          | 2/89752 [00:00<54:18, 27.54it/s]


NameError: name 'scale_eta' is not defined

In [232]:
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']

In [233]:
Reconstructed_df_np_rpc3 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)

Processing Events: 100%|██████████| 89822/89822 [00:26<00:00, 3326.75it/s]


In [92]:
Reconstructed_df_6.to_excel('Reconstructed_df_5.xlsx')

In [153]:
Reconstructed_df_3 = pd.read_excel('Reconstructed_df_3.xlsx')

In [210]:
def parse_times(row):
    coords =row['Used Coordinates']
    times = {0: [], 1: [], 2: []}

    for coord in coords:
        rpc_id = int(coord[0][3])  
        time = coord[2]
        if rpc_id in [0, 1, 2]:
            times[0].append(time)
        elif rpc_id == 3:
            times[1].append(time)
        elif rpc_id in [4, 5]:
            times[2].append(time)

    for key in times:
        if times[key]:
            times[key] = sum(times[key]) / len(times[key])
        else:
            times[key] = None  

    return times

def determine_direction(times):
    t1, t2, t3 = times[0], times[1], times[2]

    if t1 is not None and t3 is not None:
        return 'bottom' if t1 < t3 else 'top'
    elif t1 is not None and t2 is not None:
        return 'bottom' if t1 < t2 else 'top'
    elif t2 is not None and t3 is not None:
        return 'bottom' if t2 < t3 else 'top'
    else:
        return 'unknown'  

Reconstructed_df_6['Parsed Times'] = Reconstructed_df_6.apply(parse_times, axis=1)
Reconstructed_df_6['Muon Direction'] = Reconstructed_df_6['Parsed Times'].apply(determine_direction)

In [15]:
Reconstructed_df_6 = Reconstructed_df_6.sort_values(by = ['Event Number', 'Direction'])

In [16]:
valid_events = Reconstructed_df_6.groupby('Event Number')['Direction'].agg(set).reset_index()
valid_events = valid_events[valid_events['Direction'] == {'eta', 'phi'}]


# Filter the original DataFrame to include only those Event Numbers with both directions
Reconstructed_df_6_True = Reconstructed_df_6[Reconstructed_df_6['Event Number'].isin(valid_events['Event Number'])]

In [17]:
def refine_reconstructed_paths(df):
    df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)

    def find_matching_pairs(group):
        eta_rows = group[group['Direction'] == 'eta']
        phi_rows = group[group['Direction'] == 'phi']

        eta_dict = {}
        for index, row in eta_rows.iterrows():
            eta_id_list = tuple(coord[0] for coord in row['Used Coordinates'])
            if eta_id_list not in eta_dict:
                eta_dict[eta_id_list] = []
            eta_dict[eta_id_list].append(index)
        
        phi_dict = {}
        for index, row in phi_rows.iterrows():
            phi_id_list = tuple(coord[0] for coord in row['Used Coordinates'])
            if phi_id_list not in phi_dict:
                phi_dict[phi_id_list] = []
            phi_dict[phi_id_list].append(index)
        
        # Find matching pairs using dictionaries for fast lookup
        matched_indices = set()
        for eta_id_list in eta_dict:
            if eta_id_list in phi_dict:
                for eta_index in eta_dict[eta_id_list]:
                    for phi_index in phi_dict[eta_id_list]:
                        matched_indices.add(eta_index)
                        matched_indices.add(phi_index)

        return matched_indices

    # Use tqdm to add a progress bar
    all_matched_indices = set()
    for name, group in tqdm(df.groupby(['Event Number', 'Number_of_Rpcs']), desc="Processing groups"):
        matched_indices = find_matching_pairs(group)
        all_matched_indices.update(matched_indices)

    # Filter the original DataFrame to keep only rows with matched indices
    filtered_df = df.loc[all_matched_indices].reset_index(drop=True)

    return filtered_df

filtered_df = refine_reconstructed_paths(Reconstructed_df_6_True)


C:\Users\Peter\AppData\Local\Temp\ipykernel_21380\1494489405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)
Processing groups: 100%|██████████| 24680/24680 [00:27<00:00, 905.53it/s] 
C:\Users\Peter\AppData\Local\Temp\ipykernel_21380\1494489405.py:40: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_df = df.loc[all_matched_indices].reset_index(drop=True)


In [18]:
max_rpcs_df = filtered_df.groupby(['Event Number', 'Direction']).apply(
        lambda group: group[group['Number_of_Rpcs'] == group['Number_of_Rpcs'].max()]
    ).reset_index(drop=True)

In [19]:
def min_chi2(df):
    df['Number_of_Rpcs'] = df['Used Coordinates'].apply(len)
    refined_df = df.groupby(['Event Number', 'Direction']).apply(
        lambda x: x.nsmallest(1, 'chi2 norm')
    ).reset_index(drop=True)
    return refined_df
    
filtered_df_min_chi2 = min_chi2(max_rpcs_df)

In [20]:
def Final_filter(df):
    grouped = df.groupby('Event Number')

    valid_indices = []

    for name, group in grouped:
        eta_size = group[group['Direction'] == 'eta']['Number_of_Rpcs']
        phi_size = group[group['Direction'] == 'phi']['Number_of_Rpcs']
        if int(eta_size) == int(phi_size):
            valid_indices.extend(group.index) 
    return df.loc[valid_indices]

filtered_df_min_chi2 = Final_filter(filtered_df_min_chi2)


In [21]:
def optimal_pair_indices(group):
    etas = group[group['Direction'] == 'eta']
    phis = group[group['Direction'] == 'phi']

    best_eta_idx = None
    best_phi_idx = None
    min_total_diff = float('inf')

    for eta_idx, eta_row in etas.iterrows():
        for phi_idx, phi_row in phis.iterrows():
            if eta_row['Number_of_Rpcs'] != phi_row['Number_of_Rpcs']:
                continue
            eta_ids = [eta[0] for eta in eta_row['Used Coordinates']]
            phi_ids = [phi[0] for phi in phi_row['Used Coordinates']]
            if eta_ids != phi_ids:
                continue       
            total_diff = sum(abs(eta[2] - phi[2]) for eta, phi in zip(eta_row['Used Coordinates'], phi_row['Used Coordinates']))

            if total_diff < min_total_diff:
                min_total_diff = total_diff
                best_eta_idx = eta_idx
                best_phi_idx = phi_idx

    return [best_eta_idx, best_phi_idx]

optimal_indices = max_rpcs_df.groupby('Event Number').apply(optimal_pair_indices).explode().dropna().unique()

refined_df = max_rpcs_df.loc[optimal_indices]

In [ ]:
def check_rpc_conditions(coords):
    has_rpc3 = any('rpc3' == rpc for rpc, _, _ in coords)
    has_rpc4_or_rpc5 = any(rpc in ['rpc4', 'rpc5'] for rpc, _, _ in coords)
    return has_rpc3 and has_rpc4_or_rpc5

def filter_matching_rpcs(group):
    coords = group['Used Coordinates'].iloc[0] 
    if check_rpc_conditions(coords):
        return group
    return group.iloc[0:0] 

refined_df = refined_df.groupby('Event Number').apply(filter_matching_rpcs).reset_index(drop=True)
filtered_df_min_chi2 = filtered_df_min_chi2.groupby('Event Number').apply(filter_matching_rpcs).reset_index(drop=True)


In [297]:
def is_continuous(rpc_list):
    indices = [int(rpc[0][3:]) for rpc in rpc_list]
    sorted_indices = sorted(indices)
    return sorted_indices == list(range(min(sorted_indices), max(sorted_indices)+1))

filtered_df['is_continuous'] = filtered_df['Used Coordinates'].apply(is_continuous)

filtered_refined_contineous_df = filtered_df[filtered_df['is_continuous']]

filtered_refined_contineous_df.drop(columns='is_continuous', inplace=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_25604\1105133516.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [301]:
rpc_heights = {
    'rpc0': 0, 
    'rpc1': 0.5, 
    'rpc2': 1.0, 
    'rpc3': 61.5, 
    'rpc4': 121.5, 
    'rpc5': 122.0
}
    
def should_delete_row(row):
    used_rpcs = [rpc[0] for rpc in row['Used Coordinates']]
    slope = row['Slope']
    intercept = row['Intercept']
    direction = row['Direction']

    if direction == 'eta':
        bounds = (-32 * 2.5,0)
    elif direction == 'phi':
        bounds = (0, 64 * 2.5)
    else:
        raise ValueError("Unknown direction")

    for rpc, height in rpc_heights.items():
        if rpc not in used_rpcs:
            x_value = slope * height + intercept
            if bounds[0] <= x_value <= bounds[1]:
                return True 

    return False

filtered_refined_contineous_df['Delete Row'] = filtered_refined_contineous_df.apply(should_delete_row, axis=1)
filtered_refined_contineous_Deleted_df = filtered_refined_contineous_df[~filtered_refined_contineous_df['Delete Row']] 

C:\Users\Peter\AppData\Local\Temp\ipykernel_25604\2894165069.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [416]:
Reconstructed_df_6_cleaned = Reconstructed_df_6

This one best one

In [25]:
Reconstructed_df_6_cleaned = filtered_df[filtered_df['chi2 norm'] > 0.0000]
#BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST BEST

In [51]:
Reconstructed_df_6_cleaned = filtered_df

In [92]:
Reconstructed_df_6_cleaned = refined_df[refined_df['chi2 norm'] != 0]

In [94]:
Reconstructed_df_6_cleaned = refined_df

In [307]:
Reconstructed_df_6_cleaned = filtered_refined_contineous_df

In [302]:
Reconstructed_df_6_cleaned = filtered_refined_contineous_Deleted_df

In [169]:
eta_df = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'eta']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))

# angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
# angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=500, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees) from y axis')
plt.ylabel('Occurance')
plt.title('Any three RPC cross three chamber angular distribution eta direction')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [82]:
eta_df = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))

# angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
# angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.annotate(f'Number of events: {len(eta_df)}', xy=(0.75, 0.9), xycoords='axes fraction', fontsize=12, color='red')
plt.hist(angles_y_axis, bins=100, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees) from y axis')
plt.ylabel('Occurance')
plt.title('Any three RPC cross three chamber angular distribution phi direction')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [29]:
eta_df = refined_df[refined_df['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))

# angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
# angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.annotate(f'Number of events: {len(eta_df)}', xy=(0.75, 0.9), xycoords='axes fraction', fontsize=12, color='red')
plt.hist(angles_y_axis, bins=60, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees) from y axis')
plt.ylabel('Occurance')
plt.title('Any three RPC cross three chamber angular distribution phi direction')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [149]:
import csv
import math
import matplotlib.pyplot as plt
from PIL import Image

In [150]:
expFluxX = []
expFluxY = []
with open('proAnubCosFlux_lowE.csv','r') as csvfile:
    fluxReader = csv.reader(csvfile, delimiter = ',')
    for row in fluxReader:
        if float(row[0]) not in expFluxX:
            expFluxX.append(float(row[0]))
        if float(row[1]) not in expFluxY:
            expFluxY.append(float(row[1]))
expFluxZ = [[0 for y in expFluxY] for x in expFluxX]
with open('proAnubCosFlux_lowE.csv','r') as csvfile:
    fluxReader = csv.reader(csvfile, delimiter = ',')
    for row in fluxReader:
        xbin = 0
        ybin = 0
        for thisX in range(len(expFluxX)):
            if float(row[0])<=expFluxX[thisX]:
                xbin = thisX
                break
        for thisY in range(len(expFluxY)):
            if float(row[1])<=expFluxY[thisY]:
                ybin = thisY
                break        
        expFluxZ[xbin][ybin] = float(row[2])
for idx, val in enumerate(expFluxX):
    expFluxX[idx] = val-0.025
expFluxX.append(expFluxX[-1]+0.05)
#for idx, val in enumerate(expFluxY):
#    expFluxY[idx] = val-0.025
expFluxY.append(expFluxY[-1]+0.05)

In [151]:
def thetaTransform(theta,phi):
    return math.acos(-math.sin(math.pi/4.)*math.sin(theta)*math.cos(phi)+math.cos(-math.pi/4)*math.cos(theta))
def etaTransform(theta,phi):
    return math.acos(-math.sin(math.pi/4.)*math.sin(theta)*math.cos(phi)+math.cos(-math.pi/4)*math.cos(theta))
def xyyzTransform(theta,phi):
    x = math.sin(theta)*math.cos(phi)
    y = math.sin(theta)*math.sin(phi)
    z = math.cos(theta)
    xzAngle = math.atan(x/z)+math.pi/4.
    if xzAngle>math.pi/2.:
        xzAngle=xzAngle-math.pi
    yzAngle = math.atan(y/z)
    return xzAngle, yzAngle

In [152]:
totTime = 60*60
etaBins = [x*2*3.14/72.-3.14 for x in range(72)]
phiBins = [x*2*3.14/72.-3.14 for x in range(72)]
etaFluxes = [0 for xval in range(len(etaBins))]
phiFluxes = [0 for xval in range(len(phiBins))]
for idx, xval in enumerate(expFluxX[:-2]):
    sumFlux = 0
    for idy, yval in enumerate(expFluxY[:-2]):
        etaAng, phiAng = xyyzTransform(xval, yval)
        for etBin, etVal in enumerate(etaBins):
            if etVal>etaAng:
                etaFluxes[etBin] = etaFluxes[etBin]+expFluxZ[idx][idy]*math.sin(xval)*0.05*0.1*math.pi/2.
                break
        for phBin, phVal in enumerate(phiBins):
            if phVal>phiAng:
                phiFluxes[phBin] = phiFluxes[phBin]+expFluxZ[idx][idy]*math.sin(xval)*0.05*0.1*math.pi/2.
                break     

In [148]:
print(etaFluxes)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.011573692367736034, 0.010808107118785389, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.05980719866295954, 0.15521756145348486, 0.2095647211509601, 0.2158311818081606, 0.22511383237723348, 0.24821428132287537, 0.252854443656462, 0.25317122005144743, 0.23508888578935366, 0.2193938320057536, 0.1931553803799903, 0.2510079435602709, 1.3882968195814946, 1.7026370377113742, 1.1549814209455502, 0.32408909849929096, 0.14907029076025327, 0.06694392766305744, 0.030867608326048998, 0.005876743736835026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [157]:
angBins = [xval*360/2./math.pi for xval in etaBins]
fig, ax = plt.subplots(figsize=(12, 8))
thisplot = ax.step(angBins, etaFluxes,'k',where='pre',linewidth=3)
ax.set_xlabel("Angle from proANUBIS XZ Plane")
ax.set_ylabel("Muon Flux (Hz/bin)")
ax.set_xlim(-90,90)
ax.set_ylim(0,2.*totTime)
# im = image.imread('/eos/user/m/mireveri/anubis/ANUBISLogo.png')
# ax.imshow(im, aspect='auto', extent=(-75,-65, 1.7*totTime, 1.85*totTime), zorder=1)
#ax.set_yscale('log')
plt.savefig('expCosmicsXZplane.png')

# img = Image.open('expCosmicsXZplane.png')
# img = img.convert("RGBA")
# pixdata = img.load()
# width, height = img.size
# for y in range(height):
#     for x in range(width):
#         if pixdata[x, y] == (255, 255, 255, 255):
#             #Manually make any white pixel transparent so that they can stack together nicely.
#             pixdata[x, y] = (255, 255, 255, 0)
# img.save('expCosmicsXZplane.png', "PNG")

# Load the modified image for display
img_show = Image.open('expCosmicsXZplane.png')

# Display the modified image
plt.figure(figsize=(12, 8))
plt.imshow(img_show)
plt.axis('off')  # Hide axis
plt.show()

In [388]:
eta_df = Reconstructed_df_6_cleaned[Reconstructed_df_6_cleaned['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(eta_df['Slope']))

# angles_y_axis = np.where(angles_y_axis > 90, angles_y_axis - 180, angles_y_axis)
# angles_y_axis = np.where(angles_y_axis < -90, angles_y_axis + 180, angles_y_axis)


plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Any threeAngular_Distribution_phi_real_filtered')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [91]:
# Setting up the figure
plt.figure(figsize=(24, 12))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=600, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = Reconstructed_df_6[Reconstructed_df_6['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 phi at least 5 Atlas Luminous generous window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


In [90]:
# Setting up the figure
plt.figure(figsize=(12, 6))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=50, color='green', alpha=0.7)
plt.title('Minimize Chi2 Eta')
plt.xlabel('normalised Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=50, color='red', alpha=0.7)
plt.title('Minimize Chi2 phi')
plt.xlabel('normalised Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [42]:
print(filtered_df_min_chi2.info())
print(refined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2546 entries, 0 to 2545
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      2546 non-null   float64
 1   Direction         2546 non-null   object 
 2   Slope             2546 non-null   float64
 3   Intercept         2546 non-null   float64
 4   Used Coordinates  2546 non-null   object 
 5   chi2 norm         2546 non-null   float64
 6   RSS               2546 non-null   float64
 7   Number_of_Rpcs    2546 non-null   int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 159.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2294 entries, 0 to 2293
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      2294 non-null   float64
 1   Direction         2294 non-null   object 
 2   Slope             2294 non-null   float64
 3   Intercept

In [89]:
# Setting up the figure
plt.figure(figsize=(12, 6))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = refined_df[refined_df['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=50, color='green', alpha=0.7)
plt.title('Chi2 Eta Minimize Time window')
plt.xlabel('Normalised Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = refined_df[refined_df['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=50, color='red', alpha=0.7)
plt.title('Chi2 phi minimize Time window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


In [88]:
grouped_df1 = refined_df.groupby(['Event Number'])['chi2 norm'].sum().reset_index() / 2
grouped_df2 = filtered_df_min_chi2.groupby(['Event Number'])['chi2 norm'].sum().reset_index() / 2

plt.figure(figsize=(24, 12))
eta_chi2_sum1 = grouped_df1['chi2 norm']
eta_chi2_sum2 = grouped_df2['chi2 norm']
plt.hist(eta_chi2_sum2, bins=100, color='blue', alpha=0.7, label ='Minimize chi2')
plt.hist(eta_chi2_sum1, bins=100, color='red', alpha=0.7, label ='Minimize time')

plt.title('Combined Chi2')
plt.xlabel('Summed Chi2 Values')
plt.ylabel('Number')
plt.grid(True)
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

In [75]:
all_data = np.concatenate([grouped_df1['chi2 norm'], grouped_df2['chi2 norm']])
bin_edges = np.linspace(min(all_data), max(all_data), 101)  # 100 bins means 101 edges

# Plot the histograms with these bin edges
plt.figure(figsize=(12, 6))
plt.hist(grouped_df2['chi2 norm'], bins=bin_edges, color='red', alpha=0.7, label='Minimize chi2')
plt.hist(grouped_df1['chi2 norm'], bins=bin_edges, color='green', alpha=0.7, label='Minimize time')

plt.title('Combined Chi2 for at least 4 RPCs cross three chambers')
plt.xlabel('Summed Chi2 Values / 2')
plt.ylabel('Number')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

KeyboardInterrupt: 

In [120]:
show(filtered_df_min_chi2)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [40]:
# Setting up the figure
plt.figure(figsize=(12, 6))

# Histogram for 'eta' direction
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
eta_chi2 = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=100, color='green', alpha=0.7)
plt.title('Chi2 Eta Minimize Time window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Histogram for 'phi' direction
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
phi_chi2 = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=100, color='red', alpha=0.7)
plt.title('Chi2 phi minimize Time window')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()


In [121]:
grouped_df = filtered_df_min_chi2.groupby(['Event Number'])['chi2 norm'].sum().reset_index()

plt.figure(figsize=(24, 12))
eta_chi2_sum = grouped_df['chi2 norm']
plt.hist(eta_chi2_sum, bins=100, color='green', alpha=0.7)
plt.title('Combined Chi2')
plt.xlabel('Summed Chi2 Values')
plt.ylabel('Number')
plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [93]:
def plot_rpc_with_vertical_line(plate_lines, vertical_line_coords):

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x_count = 32
    y_count = 64
    z_positions = [0, 10, 20] 

    for i, z in enumerate(z_positions):
        x_lines, y_lines = plate_lines.get(i, ([], [])) 
        for x in range(x_count):
            for y in range(y_count):
                color = 'gray'
                if x in x_lines:
                    color = 'blue'
                if y in y_lines:
                    color = 'green' 
                ax.bar3d(x, y, z, 1, 1, 0.5, color=color, alpha=0.5)

    vx, vy = vertical_line_coords
    vline_z = np.linspace(min(z_positions), max(z_positions) + 10, 100)
    vline_x = np.full_like(vline_z, vx)
    vline_y = np.full_like(vline_z, vy)
    vertical_line = ax.plot(vline_x, vline_y, vline_z, color='gold', linewidth=5, linestyle='-', label='Reconstructed Hits')

    ax.text(vx, vy, max(z_positions) + 5, "Reconstructed Hits", color='gold')

    # Labels and title
    ax.set_xlabel('Eta')
    ax.set_ylabel('Phi')
    ax.set_zlabel('Height (Z)')
    ax.set_title('Chi2 method of filtering')

    # Legend
    ax.legend()

    plt.show()

plate_lines = {
    0: ([5, 6], [10, 11]),      
    1: ([5, 6], [10, 11]),     
    2: ([5, 6, 12, 13], [10, 11]) #
}
vertical_line_coords = (5.5, 10.11) 

plot_rpc_with_vertical_line(plate_lines, vertical_line_coords)


Timing for triplet

In [262]:
show(filtered_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [257]:
eta_refined_df = filtered_df_min_chi2[filtered_df_min_chi2['Direction'] == 'phi']

In [132]:
show(eta_refined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [259]:
def find_triplet_time_range(rpc_list):

    rpc_names = [x[0] for x in rpc_list]
    times = {x[0]: x[2] for x in rpc_list}

    required_rpcs = ['rpc0', 'rpc1', 'rpc2']
    if all(rpc in rpc_names for rpc in required_rpcs):

        time_values = [times[rpc] for rpc in required_rpcs]
        time_range = max(time_values) - min(time_values)
        mean_time = sum(time_values) / len(time_values)
        return (True, time_range, mean_time)
    else:
        return (False, None, None)
    
eta_refined_df['triplet range'] = eta_refined_df['Used Coordinates'].apply(find_triplet_time_range)

C:\Users\Peter\AppData\Local\Temp\ipykernel_60340\4050136835.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [260]:
def find_triplet_time_range(rpc_list):

    rpc_names = [x[0] for x in rpc_list]
    times = {x[0]: x[2] for x in rpc_list}

    required_rpcs = ['rpc0', 'rpc1', 'rpc2']
    if all(rpc in rpc_names for rpc in required_rpcs):

        time_values = [times[rpc] for rpc in required_rpcs]
        top_range = times['rpc2'] - times['rpc1']
        bot_range = times['rpc1'] - times['rpc0']
        return (True, top_range, bot_range)
    else:
        return (False, None, None)
    
eta_refined_df['triplet range true'] = eta_refined_df['Used Coordinates'].apply(find_triplet_time_range)

C:\Users\Peter\AppData\Local\Temp\ipykernel_60340\1324923274.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [261]:
times = []
for k in eta_refined_df['triplet range']:
    times.append(k[1])
times = [x for x in times if x is not None]

In [228]:
plt.figure(figsize=(12, 6))

Triplet_range = eta_refined_df['triplet range']
plt.hist(times, bins=10, color='green', alpha=0.7, density=True)
plt.title('Triplet timing range for phi cluster = 1')
plt.xlabel('range/ns')
plt.ylabel('Number')

plt.grid(True)

plt.tight_layout()
plt.show()


In [262]:
timestop = []
for k in eta_refined_df['triplet range true']:
    timestop.append(k[1])
timestop = [x for x in timestop if x is not None]

timesbot = []
for k in eta_refined_df['triplet range true']:
    timesbot.append(k[2])
timesbot = [x for x in timesbot if x is not None]

print(times)

[23, 2, 1, 3, 10, 4, 2, 1, 2, 6, 1, 1, 3, 2, 3, 3, 14, 2, 5, 2, 1, 5, 3, 1, 4, 12, 23, 2, 20, 2, 1, 2, 5, 9, 1, 28, 4, 6, 1, 28, 2, 1, 2, 1, 2, 5, 2, 4, 3, 2, 4, 3, 4, 2, 3, 3, 4, 2, 1, 1, 2, 3, 26, 2, 3, 2, 3, 6, 3, 3, 26, 4, 2, 2, 2, 2, 2, 3, 5, 2, 6, 2, 2, 1, 3, 4, 3, 1, 1, 2, 3, 19, 2, 25, 1, 0, 2, 3, 1, 1, 4, 4, 2, 2, 30, 26, 2, 2, 25, 22, 1, 2, 2, 4, 3, 2, 3, 2, 3, 5, 3, 2, 10, 1, 4, 3, 5, 4, 3, 22, 2, 1, 2, 1, 24, 3, 7, 4, 0, 2, 25, 3, 1, 3, 20, 3, 3, 6, 4, 0, 5, 8, 26, 24, 0, 18, 3, 1, 28, 5, 4, 2, 2, 2, 3, 2, 2, 2, 3, 1, 3, 15, 2, 7, 4, 5, 1, 4, 6, 2, 3, 2, 2, 1, 5, 16, 3, 1, 1, 3, 4, 3, 5, 3, 4, 2, 2, 3, 5, 4, 26, 1, 2, 3, 4, 13, 2, 0, 3, 4, 4, 2, 1, 1, 4, 4, 4, 4, 2, 1, 4, 4, 4, 15, 7, 1, 6, 4, 10, 2, 2, 5, 5, 1, 3, 4, 3, 3, 2, 3, 5, 28, 3, 2, 3, 2, 1, 2, 24, 6, 4, 4, 2, 19, 1, 3, 1, 4, 2, 4, 1, 2, 1, 3, 12, 3, 1, 4, 22, 3, 25, 3, 1, 1, 1, 2, 3, 4, 1, 5, 2, 3, 18, 30, 4, 4, 1, 3, 0, 21, 1, 5, 4, 3, 2, 2, 3, 4, 3, 0, 2, 0, 6, 1, 3, 3, 5, 7, 4, 24, 4, 4, 1, 3, 1, 3, 2, 2, 4, 3

In [213]:
plt.figure(figsize=(12, 6))
binedge = np.linspace(-5, 5, 11)
# plt.xlim(-10, 10)

Triplet_range = eta_refined_df['triplet range']
plt.hist(timestop, bins=binedge, color='Salmon', alpha=0.7, label = 'Top - Mid', density=True)
plt.hist(timesbot, bins=binedge, color='green', alpha=0.7, label = 'Mid - Bot', density=True)
plt.title('Triplet timing range phi eta')
plt.xlabel('range/ns')
plt.ylabel('Number')
plt.legend()

plt.grid(True)

plt.tight_layout()
plt.show()


In [263]:
event_time_triplet = []
for k in eta_refined_df['triplet range']:
    event_time_triplet.append(k[2])
event_time_triplet = [x for x in event_time_triplet if x is not None]

print(event_time_triplet)

[261.3333333333333, 239.66666666666666, 241.66666666666666, 238.0, 250.66666666666666, 229.33333333333334, 242.0, 229.66666666666666, 232.66666666666666, 238.66666666666666, 221.33333333333334, 244.33333333333334, 245.33333333333334, 249.0, 225.33333333333334, 241.33333333333334, 303.0, 240.0, 234.33333333333334, 226.0, 225.66666666666666, 245.33333333333334, 242.33333333333334, 258.3333333333333, 240.66666666666666, 241.33333333333334, 237.0, 230.66666666666666, 239.0, 241.0, 241.66666666666666, 245.33333333333334, 245.33333333333334, 240.0, 248.66666666666666, 242.33333333333334, 248.66666666666666, 244.0, 243.33333333333334, 252.66666666666666, 252.0, 244.33333333333334, 231.0, 240.33333333333334, 231.66666666666666, 257.3333333333333, 228.0, 238.0, 248.0, 222.66666666666666, 221.66666666666666, 239.66666666666666, 254.66666666666666, 245.0, 234.66666666666666, 236.66666666666666, 243.0, 227.33333333333334, 248.33333333333334, 252.66666666666666, 242.33333333333334, 256.333333333333

In [215]:
plt.figure(figsize=(12, 6))

plt.hist(event_time_triplet, bins=20, color='green', alpha=0.7)
plt.title('Triplet event time for all 6 rpc muons eta')
plt.xlabel('range/ns')
plt.ylabel('Number')

plt.grid(True)

plt.tight_layout()
plt.show()


In [162]:
show(eta_refined_df['Used Coordinates'])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [165]:
m = [('rpc0', 5, 242), ('rpc1', 7, 243), ('rpc2', 9, 241)]
print(Bunching_FEB_Numbers(m))

(False, None)


In [264]:
def Bunching_FEB_Numbers(rpc_list):
    rpc_names = [x[0] for x in rpc_list]

    required_rpcs = ['rpc0', 'rpc1', 'rpc2']
    if all(rpc in rpc_names for rpc in required_rpcs):
        filtered_rpc_list = [item for item in rpc_list if item[0] in required_rpcs]
        
        group_numbers = set((item[1] //8) for item in filtered_rpc_list)
        unique_groups = set(group_numbers)
        all_same_group = len(unique_groups) == 1
        return (all_same_group, unique_groups)
    else:
        return (False, None)
    
eta_refined_df['FEB group triplet'] = eta_refined_df['Used Coordinates'].apply(Bunching_FEB_Numbers)

C:\Users\Peter\AppData\Local\Temp\ipykernel_60340\1971574157.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [234]:
show(eta_refined_df.head(200))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [243]:
plt.figure(figsize=(18, 9))
colors = [
    "#1f77b4",  # medium blue
    "#ff7f0e",  # vivid orange
    "#2ca02c",  # grass green
    "#d62728",  # strong red
    "#9467bd",  # muted purple
    "#8c564b",  # dark brown
    "#e377c2",  # soft pink
    "#7f7f7f",  # medium gray
    "#bcbd22",  # lime green
    "#17becf"   # cyan-teal
]
binedge = np.linspace(-5, 5, 11)
for i in range(6):
    FEB_grouped_Triplet = eta_refined_df[eta_refined_df['FEB group triplet'] == (True, {-i})]
    if FEB_grouped_Triplet.empty:
        continue

    timestop = []
    for k in FEB_grouped_Triplet['triplet range true']:
        timestop.append(k[1])
    timestop = [x for x in timestop if x is not None]
    meantop = "%.3E" %np.mean(timestop)

    timesbot = []
    for k in FEB_grouped_Triplet['triplet range true']:
        timesbot.append(k[2])
    timesbot = [x for x in timesbot if x is not None]
    meanbot = "%.3E" %np.mean(timesbot)
    times = []
    for k in FEB_grouped_Triplet['triplet range']:
        times.append(k[1])
    times = [x for x in times if x is not None]
    mean = "%.3E" % np.mean(times)
    plt.hist(timestop, bins=binedge, alpha=0.7, label = f'FEB{i} Top - Mid, mean:{meantop}', density=True, histtype='step', linewidth=3, linestyle='dashed', color = colors[i])
    plt.hist(timesbot, bins=binedge, alpha=0.7, label = f'FEB{i} Mid - Bot, mean:{meanbot}', density=True, histtype='step', linewidth=3, linestyle='solid', color = colors[i])
    # plt.hist(times, bins=30, alpha=0.9, label = f'Maximum time difference in FEB{i}, mean{mean}', density=True, histtype='step', linewidth = 2.5)
plt.title('Triplet timing separated by FEBs')
plt.xlabel('range/ns')
plt.ylabel('Number')
plt.legend()

plt.grid(True)
# Display the plots
plt.tight_layout()
plt.show()


In [270]:
plt.figure(figsize=(20, 10))
colors = [
    "#1f77b4",  # medium blue
    "#ff7f0e",  # vivid orange
    "#2ca02c",  # grass green
    "#d62728",  # strong red
    "#9467bd",  # muted purple
    "#8c564b",  # dark brown
    "#e377c2",  # soft pink
    "#7f7f7f",  # medium gray
    "#bcbd22",  # lime green
    "#17becf"   # cyan-teal
]
binedge = np.linspace(-5, 5, 11)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

for i in range(4):
    FEB_grouped_Triplet = eta_refined_df[eta_refined_df['FEB group triplet'] == (True, {i})]
    if FEB_grouped_Triplet.empty:
        continue

    timestop = [k[1] for k in FEB_grouped_Triplet['triplet range true'] if k[1] is not None]
    meantop = "%.3E" % np.mean(timestop)

    timesbot = [k[2] for k in FEB_grouped_Triplet['triplet range true'] if k[2] is not None]
    meanbot = "%.3E" % np.mean(timesbot)

    ax1.hist(timestop, bins=binedge, alpha=0.7, label=f'FEB{i} Top - Mid, mean: {meantop}', density=True, histtype='step', linewidth=2, linestyle='dashed', color=colors[i])

    ax2.hist(timesbot, bins=binedge, alpha=0.7, label=f'FEB{i} Mid - Bot, mean: {meanbot}', density=True, histtype='step', linewidth=2, linestyle='solid', color=colors[i])

ax1.set_title('Top - Mid Timing phi1 side', fontsize=16)
ax1.set_xlabel('Range/ns', fontsize=14)
ax1.set_ylabel('Density', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True)

ax2.set_title('Mid - Bot Timing phi1 side', fontsize=16)
ax2.set_xlabel('Range/ns', fontsize=14)
ax2.set_ylabel('Density', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True)

plt.tight_layout()
plt.show()

In [272]:
plt.figure(figsize=(20, 10))
colors = [
    "#1f77b4",  # medium blue
    "#ff7f0e",  # vivid orange
    "#2ca02c",  # grass green
    "#d62728",  # strong red
    "#9467bd",  # muted purple
    "#8c564b",  # dark brown
    "#e377c2",  # soft pink
    "#7f7f7f",  # medium gray
    "#bcbd22",  # lime green
    "#17becf"   # cyan-teal
]
binedge = np.linspace(-5, 5, 11)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

for i in range(10):
    if i <=3:
        continue
    FEB_grouped_Triplet = eta_refined_df[eta_refined_df['FEB group triplet'] == (True, {i})]
    if FEB_grouped_Triplet.empty:

    timestop = [k[1] for k in FEB_grouped_Triplet['triplet range true'] if k[1] is not None]
    meantop = "%.3E" % np.mean(timestop)

    timesbot = [k[2] for k in FEB_grouped_Triplet['triplet range true'] if k[2] is not None]
    meanbot = "%.3E" % np.mean(timesbot)

    ax1.hist(timestop, bins=binedge, alpha=0.7, label=f'FEB{i} Top - Mid, mean: {meantop}', density=True, histtype='step', linewidth=2, linestyle='dashed', color=colors[i])
    ax2.hist(timesbot, bins=binedge, alpha=0.7, label=f'FEB{i} Mid - Bot, mean: {meanbot}', density=True, histtype='step', linewidth=2, linestyle='solid', color=colors[i])

ax1.set_title('Top - Mid Timing phi2 side', fontsize=16)
ax1.set_xlabel('Range/ns', fontsize=14)
ax1.set_ylabel('Density', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True)

ax2.set_title('Mid - Bot Timing phi2 side', fontsize=16)
ax2.set_xlabel('Range/ns', fontsize=14)
ax2.set_ylabel('Density', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True)

plt.tight_layout()
plt.show()

In [240]:
plt.figure(figsize=(18, 9))
colors = [
    "#1f77b4",  # medium blue
    "#ff7f0e",  # vivid orange
    "#2ca02c",  # grass green
    "#d62728",  # strong red
    "#9467bd",  # muted purple
    "#8c564b",  # dark brown
    "#e377c2",  # soft pink
    "#7f7f7f",  # medium gray
    "#bcbd22",  # lime green
    "#17becf"   # cyan-teal
]
for i in range(9):
    FEB_grouped_Triplet = eta_refined_df[eta_refined_df['FEB group triplet'].apply(lambda x: x[0] == False and x[1] is not None)]
    if FEB_grouped_Triplet.empty:
        continue

    timestop = []
    for k in FEB_grouped_Triplet['triplet range true']:
        timestop.append(k[1])
    timestop = [x for x in timestop if x is not None]
    meantop = "%.3E" %np.mean(timestop)

    timesbot = []
    for k in FEB_grouped_Triplet['triplet range true']:
        timesbot.append(k[2])
    timesbot = [x for x in timesbot if x is not None]
    meanbot = "%.3E" %np.mean(timesbot)
    times = []
    for k in FEB_grouped_Triplet['triplet range']:
        times.append(k[1])
    times = [x for x in times if x is not None]
    mean = "%.3E" % np.mean(times)
    # plt.hist(timestop, bins=20, alpha=0.7, label = f'FEB{i} Top - Mid, mean:{meantop}', density=True, histtype='step', linewidth=3, linestyle='dashed', color = colors[i])
    # plt.hist(timesbot, bins=20, alpha=0.7, label = f'FEB{i} Mid - Bot, mean:{meanbot}', density=True, histtype='step', linewidth=3, linestyle='solid', color = colors[i])
    plt.hist(times, bins=10, alpha=0.9, label = f'Maximum time difference in FEB{i}, mean{mean}', density=True, histtype='step', linewidth = 2.5)
plt.title('Triplet timing separated by FEBs')
plt.xlabel('range/ns')
plt.ylabel('Number')
plt.legend()

plt.grid(True)
# Display the plots
plt.tight_layout()
plt.show()


In [169]:
FEB_grouped_Triplet = eta_refined_df[eta_refined_df['FEB group triplet'].apply(lambda x: x[0] == False and x[1] is not None)]
show(FEB_grouped_Triplet)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Cluster timing

In [66]:
Triplet_cluster_df = fully_massaged_df[fully_massaged_df['rpc_number'].isin(['rpc0', 'rpc1', 'rpc2'])]

In [11]:
Triplet_cluster_df = cluster_df[cluster_df['rpc_number'].isin(['rpc0', 'rpc1', 'rpc2'])]

In [74]:

cluster_df['time_diff'] = cluster_df['times'].apply(lambda x: max(x) - min(x))

min_time_diff = cluster_df['time_diff'].min()
max_time_diff = cluster_df['time_diff'].max()

bins = np.linspace(0, 12, 12) 

plt.figure(figsize=(12, 8))
plt.xlim(0, 20)

bin_edges = np.arange(0,21,1)

for i in range(10):
    if i == 0 or i == 1:
        continue
    size_time_diff = cluster_df[cluster_df['size'] == i]['time_diff']
    plt.hist(size_time_diff, bins=bin_edges, alpha=0.7, label=f'size {i}', histtype='step', linewidth=2)

plt.title('Time Difference for Different Cluster Sizes')
plt.xlabel('Time Difference (nanoseconds)')
plt.ylabel('Frequency')
plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

In [72]:
bin_edges = np.arange(-0.5, 10, 1)
plt.figure(figsize=(12, 8))
plt.xlim(0, 9)
plt.hist(cluster_df['size'] , bins=bin_edges, alpha=0.7)

plt.title('cluster sizes distribution')
plt.xlabel('sizes')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

In [67]:
show(Triplet_cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [371]:
print(Triplet_cluster_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580577 entries, 0 to 759898
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     580577 non-null  int64 
 1   rpc_number       580577 non-null  object
 2   strip_direction  580577 non-null  object
 3   locations        580577 non-null  object
 4   times            580577 non-null  object
 5   start_time       580577 non-null  int64 
 6   end_time         580577 non-null  int64 
 7   size             580577 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 39.9+ MB
None


In [68]:
Triplet_cluster_df_filtered = Triplet_cluster_df[Triplet_cluster_df['size'] > 1]

Triplet_cluster_df_filtered['time_diff'] = Triplet_cluster_df_filtered['times'].apply(lambda x: max(x) - min(x))

rpc0_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc0']
rpc1_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc1']
rpc2_data = Triplet_cluster_df_filtered[Triplet_cluster_df_filtered['rpc_number'] == 'rpc2']

plt.figure(figsize=(12, 8))
plt.hist(rpc0_data['time_diff'], bins=20, alpha=0.7, color='red', edgecolor='red', label='RPC0', histtype='step', linewidth=2)
plt.hist(rpc1_data['time_diff'], bins=20, alpha=0.7, color='green', edgecolor='green', label='RPC1', histtype='step', linewidth=2)
plt.hist(rpc2_data['time_diff'], bins=20, alpha=0.7, color='blue', edgecolor='blue', label='RPC2', histtype='step', linewidth=2)

plt.title('Cluster Max-Min Times Difference in Triplet for Each RPC Number')
plt.xlabel('Time Difference (nanoseconds)')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

In [69]:

Triplet_cluster_df_filtered = Triplet_cluster_df[Triplet_cluster_df['size'] > 1]

Triplet_cluster_df_filtered['time_diff'] = Triplet_cluster_df_filtered['times'].apply(lambda x: max(x) - min(x))

plt.figure(figsize=(12, 8))

colors = ['red', 'green', 'blue']
styles = ['-', '--', ':']
rpc_numbers = Triplet_cluster_df_filtered['rpc_number'].unique()
sizes = Triplet_cluster_df_filtered['size'].unique()

bin = [6,8,10,12,14]

for size_index, size in enumerate(sorted(sizes)):
    if size <=5:
        data = Triplet_cluster_df_filtered[
            (Triplet_cluster_df_filtered['size'] == size)
        ]

        # Only plot if there is data
        if not data.empty:
            plt.hist(data['time_diff'], bins=bin[size_index], alpha=0.7, label=f'Size {size}', histtype='step', linewidth=2)

# Plot customization
plt.title('Cluster Max-Min Times Difference in Triplet for Each RPC Number and Size')
plt.xlabel('Time Difference (nanoseconds)')
plt.ylabel('Frequency')
plt.grid(True)
plt.yscale('log')
plt.legend(title='RPC and Size')
plt.show()

In [375]:
show(Triplet_cluster_df_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [137]:
print(Reconstructed_df_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88254 entries, 0 to 88253
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      88254 non-null  int64  
 1   Direction         88254 non-null  object 
 2   Slope             88254 non-null  float64
 3   Intercept         88254 non-null  float64
 4   Used Coordinates  88254 non-null  object 
 5   chi2 norm         88254 non-null  float64
 6   Number_of_Rpcs    88254 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 4.7+ MB
None


In [167]:
show(Reconstructed_df_4)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [190]:
show(Reconstructed_df_3)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [175]:
print(Reconstructed_df_4['Used Coordinates'])

0        [(rpc0, 51, 250.04666666666665), (rpc1, 51, 24...
1        [(rpc0, 22, 237.38), (rpc1, 22, 237.79), (rpc2...
2        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
3        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
4        [(rpc0, 49, 244.38), (rpc1, 49, 243.79), (rpc2...
                               ...                        
88249    [(rpc0, 3, 231.38), (rpc1, 3, 230.79), (rpc2, ...
88250    [(rpc0, 3, 231.38), (rpc1, 3, 230.79), (rpc2, ...
88251    [(rpc0, 56, 280.38), (rpc1, 56, 269.79), (rpc2...
88252    [(rpc0, -27, 255.06), (rpc1, -27, 256.64), (rp...
88253    [(rpc0, -27, 255.06), (rpc1, -27, 256.64), (rp...
Name: Used Coordinates, Length: 88254, dtype: object


In [186]:
def convert_string_to_tuples(data):
    try:
        # Convert the string representation back to the actual data type (list of tuples)
        return ast.literal_eval(data)
    except (SyntaxError, ValueError) as e:
        print(f"Error converting data: {data}. Error: {e}")
        return None
    
Reconstructed_df_3['Used Coordinates'] = Reconstructed_df_3['Used Coordinates'].apply(convert_string_to_tuples)

In [196]:
def check_rpc_conditions(coords):
    has_rpc3 = any('rpc3' == rpc for rpc, _, _ in coords)
    has_rpc4_or_rpc5 = any(rpc in ['rpc4', 'rpc5'] for rpc, _, _ in coords)
    return has_rpc3 and has_rpc4_or_rpc5

def extract_rpc_set(coords):
    return set(rpc for rpc, _, _ in coords)

def filter_matching_rpcs(group):
    if 'eta' in group['Direction'].values and 'phi' in group['Direction'].values:
        eta_rpcs = group[group['Direction'] == 'eta']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        phi_rpcs = group[group['Direction'] == 'phi']['Used Coordinates'].apply(extract_rpc_set).iloc[0]
        if eta_rpcs == phi_rpcs:
            return group
    return pd.DataFrame() 

Reconstructed_df_3 = Reconstructed_df_3.groupby('Event Number').apply(filter_matching_rpcs).reset_index(drop=True)

Cross_condition_df = Reconstructed_df_3[Reconstructed_df_3['Used Coordinates'].apply(check_rpc_conditions)]
Cross_condition_df = refine_reconstructed_paths(Cross_condition_df)
valid_events = Cross_condition_df.groupby('Event Number')['Direction'].agg(set).reset_index()
valid_events = valid_events[valid_events['Direction'] == {'eta', 'phi'}]

Cross_condition_df = Cross_condition_df[Cross_condition_df['Event Number'].isin(valid_events['Event Number'])]

C:\Users\Peter\AppData\Local\Temp\ipykernel_40476\1569048960.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [200]:
show(Reconstructed_df_3)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [202]:

plt.figure(figsize=(24, 12))

plt.subplot(1, 2, 1) 
eta_chi2 = Cross_condition_df[Cross_condition_df['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

plt.subplot(1, 2, 2) 
phi_chi2 = Cross_condition_df[Cross_condition_df['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 Phi at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

# Display the plots
plt.tight_layout()
plt.show()

In [201]:

plt.figure(figsize=(24, 12))

plt.subplot(1, 2, 1) 
eta_chi2 = Reconstructed_df_3[Reconstructed_df_3['Direction'] == 'eta']['chi2 norm']
plt.hist(eta_chi2, bins=60, color='green', alpha=0.7)
plt.title('Chi2 Eta at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

plt.subplot(1, 2, 2) 
phi_chi2 = Reconstructed_df_3[Reconstructed_df_3['Direction'] == 'phi']['chi2 norm']
plt.hist(phi_chi2, bins=60, color='red', alpha=0.7)
plt.title('Chi2 Phi at least 3 three chamber Atlas ON')
plt.xlabel('Chi2 Values')
plt.ylabel('Number')

plt.grid(True)

plt.tight_layout()
plt.show()

In [87]:
Reconstructed_df_6 = Reconstructed_df_6.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
              .reset_index(drop=True)

In [23]:
print(Reconstructed_df_6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      964 non-null    int64  
 1   Direction         964 non-null    object 
 2   Slope             964 non-null    float64
 3   Intercept         964 non-null    float64
 4   Slope_error       964 non-null    int64  
 5   Intercept_error   964 non-null    int64  
 6   Used Coordinates  964 non-null    object 
 7   RSS               964 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 60.4+ KB
None


In [59]:
print(len([1,2,3,4]))

4


In [22]:
show(Reconstructed_df_6)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [49]:
# var_eta =  ( 1 * 3.09375) ** 2 / 12
# var_phi = ( 1 * 2.8125) ** 2 / 12
# rpc_heights = {
#     'rpc0': 0, 
#     'rpc1': 0.5, 
#     'rpc2': 1.0, 
#     'rpc3': 61.5, 
#     'rpc4': 121.5, 
#     'rpc5': 122.0
# }

In [109]:

# scale_eta = 3.09375
# scale_phi = 2.8125

# def parse_and_scale_coordinates(used_coords, direction):
#     if isinstance(used_coords, str):
#         coords = ast.literal_eval(used_coords)
#     else:
#         coords = used_coords

#     scale_factor = scale_eta if direction == 'eta' else scale_phi

#     y_vals = [rpc_heights[coord[0]] for coord in coords]
#     x_vals = [coord[1] * scale_factor for coord in coords]
#     return np.array(x_vals), np.array(y_vals)

# Reconstructed_df_6[['x', 'y']] = Reconstructed_df_6.apply(
#     lambda row: pd.Series(parse_and_scale_coordinates(row['Used Coordinates'], row['Direction']), index=['x', 'y']),
#     axis=1
# )

# def calculate_chi_squared(row):
#     x = np.array(row['x'])
#     y = np.array(row['y'])
#     a = row['Slope']
#     b = row['Intercept']
#     x_pred = a * y + b

#     if row['Direction'] == 'eta':
#         sigma_squared = var_eta
#     elif row['Direction'] == 'phi':
#         sigma_squared = var_phi
#     else:
#         sigma_squared = 1  
#     chi_squared = np.sum(((x - x_pred) ** 2) / sigma_squared)
#     chi_squared_norm = chi_squared / (row['Number_of_Rpcs'] - 2)
#     return chi_squared_norm

# Reconstructed_df_6['Chi2'] = Reconstructed_df_6.apply(calculate_chi_squared, axis=1)


In [128]:
df_no_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc0']
df_no_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc1']
df_no_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc2']
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']
df_no_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc4']
df_no_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc5']


In [ ]:
df_recon_no_rpc0_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc0, 5)
df_recon_no_rpc1_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc1, 5)
df_recon_no_rpc2_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc2, 5)
df_recon_no_rpc3_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)
df_recon_no_rpc4_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc4, 5)
df_recon_no_rpc5_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc5, 5)

Processing Events: 100%|██████████| 92919/92919 [01:17<00:00, 1196.73it/s]


In [ ]:
df_recon_no_rpc0_4.to_excel('df_recon_no_rpc0_5_AtlasOFF.xlsx')
df_recon_no_rpc1_4.to_excel('df_recon_no_rpc1_5_AtlasOFF.xlsx')
df_recon_no_rpc2_4.to_excel('df_recon_no_rpc2_5_AtlasOFF.xlsx')
df_recon_no_rpc3_4.to_excel('df_recon_no_rpc3_5_AtlasOFF.xlsx')
df_recon_no_rpc4_4.to_excel('df_recon_no_rpc4_5_AtlasOFF.xlsx')
df_recon_no_rpc5_4.to_excel('df_recon_no_rpc5_5_AtlasOFF.xlsx')

In [129]:
df_recon_no_rpc0_4 = pd.read_excel('df_recon_no_rpc0_5_AtlasON.xlsx')
df_recon_no_rpc1_4 = pd.read_excel('df_recon_no_rpc1_5_AtlasON.xlsx')
df_recon_no_rpc2_4 = pd.read_excel('df_recon_no_rpc2_5_AtlasON.xlsx')
df_recon_no_rpc3_4 = pd.read_excel('df_recon_no_rpc3_5_AtlasON.xlsx')
df_recon_no_rpc4_4 = pd.read_excel('df_recon_no_rpc4_5_AtlasON.xlsx')
df_recon_no_rpc5_4 = pd.read_excel('df_recon_no_rpc5_5_AtlasON.xlsx')

In [129]:
print(df_recon_no_rpc0_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Event Number      1253 non-null   int64  
 1   Direction         1253 non-null   object 
 2   Slope             1253 non-null   float64
 3   Intercept         1253 non-null   float64
 4   Slope_error       1253 non-null   int64  
 5   Intercept_error   1253 non-null   int64  
 6   Used Coordinates  1253 non-null   object 
 7   RSS               1253 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 78.4+ KB
None


In [75]:
show(df_recon_no_rpc0_4.sort_values('Event Number'))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [130]:
df_recon_no_rpc0_4_filtered = df_recon_no_rpc0_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
              .reset_index(drop=True)
              
df_recon_no_rpc1_4_filtered = df_recon_no_rpc1_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS')) \
                                .reset_index(drop=True)
df_recon_no_rpc2_4_filtered = df_recon_no_rpc2_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
              
df_recon_no_rpc3_4_filtered = df_recon_no_rpc3_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc4_4_filtered = df_recon_no_rpc4_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)
df_recon_no_rpc5_4_filtered = df_recon_no_rpc5_4.groupby(['Event Number', 'Direction'], as_index=False, group_keys=False) \
              .apply(lambda x: x if len(x) == 1 else x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction'], as_index=False) \
              .apply(lambda x: x.nsmallest(1, 'RSS'))\
                                .reset_index(drop=True)

In [74]:
show(df_recon_no_rpc0_4_filtered.reset_index(drop=True))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [131]:
df_recon_no_rpc0_4_filtered['expected hits'] = df_recon_no_rpc0_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc1_4_filtered['expected hits'] = df_recon_no_rpc1_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc2_4_filtered['expected hits'] = df_recon_no_rpc2_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc3_4_filtered['expected hits'] = df_recon_no_rpc3_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc4_4_filtered['expected hits'] = df_recon_no_rpc4_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc5_4_filtered['expected hits'] = df_recon_no_rpc5_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)

In [132]:
fully_massaged_df_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc0']
fully_massaged_df_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc1']
fully_massaged_df_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc2']
fully_massaged_df_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc3']
fully_massaged_df_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc4']
fully_massaged_df_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc5']

def calculate_midpoint(locations, direction):
    if direction == 'eta':
        return np.mean([loc[1] for loc in locations])
    elif direction == 'phi':
        return np.mean([loc[0] for loc in locations])
    return None

fully_massaged_df_rpc0['midpoint'] = fully_massaged_df_rpc0.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc1['midpoint'] = fully_massaged_df_rpc1.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc2['midpoint'] = fully_massaged_df_rpc2.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc3['midpoint'] = fully_massaged_df_rpc3.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc4['midpoint'] = fully_massaged_df_rpc4.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc5['midpoint'] = fully_massaged_df_rpc5.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)

merged_df0 = pd.merge(df_recon_no_rpc0_4_filtered, fully_massaged_df_rpc0, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df1 = pd.merge(df_recon_no_rpc1_4_filtered, fully_massaged_df_rpc1, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df2 = pd.merge(df_recon_no_rpc2_4_filtered, fully_massaged_df_rpc2, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df3 = pd.merge(df_recon_no_rpc3_4_filtered, fully_massaged_df_rpc3, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df4 = pd.merge(df_recon_no_rpc4_4_filtered, fully_massaged_df_rpc4, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df5 = pd.merge(df_recon_no_rpc5_4_filtered, fully_massaged_df_rpc5, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')


merged_df0['hit_difference'] = merged_df0['expected hits'] - merged_df0['midpoint']
merged_df1['hit_difference'] = merged_df1['expected hits'] - merged_df1['midpoint']
merged_df2['hit_difference'] = merged_df2['expected hits'] - merged_df2['midpoint']
merged_df3['hit_difference'] = merged_df3['expected hits'] - merged_df3['midpoint']
merged_df4['hit_difference'] = merged_df4['expected hits'] - merged_df4['midpoint']
merged_df5['hit_difference'] = merged_df5['expected hits'] - merged_df5['midpoint']

result_df0 = merged_df0[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df1 = merged_df1[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df2 = merged_df2[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df3 = merged_df3[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df4 = merged_df4[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df5 = merged_df5[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]


C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\2424636361.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [133]:
eta_df0 = result_df0[result_df0['Direction'] == 'eta']
eta_df1 = result_df1[result_df1['Direction'] == 'eta']
eta_df2 = result_df2[result_df2['Direction'] == 'eta']
eta_df3 = result_df3[result_df3['Direction'] == 'eta']
eta_df4 = result_df4[result_df4['Direction'] == 'eta']
eta_df5 = result_df5[result_df5['Direction'] == 'eta']

phi_df0 = result_df0[result_df0['Direction'] == 'phi']
phi_df1 = result_df1[result_df1['Direction'] == 'phi']
phi_df2 = result_df2[result_df2['Direction'] == 'phi']
phi_df3 = result_df3[result_df3['Direction'] == 'phi']
phi_df4 = result_df4[result_df4['Direction'] == 'phi']
phi_df5 = result_df5[result_df5['Direction'] == 'phi']

In [134]:
def get_min_index(group):
    if group['hit_difference'].isna().all():
        return group.index.tolist() 
    return [group['hit_difference'].idxmin()]


In [135]:
phi_df0 = phi_df0.loc[phi_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df1 = phi_df1.loc[phi_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df2 = phi_df2.loc[phi_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df3 = phi_df3.loc[phi_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df4 = phi_df4.loc[phi_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df5 = phi_df5.loc[phi_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]

eta_df0 = eta_df0.loc[eta_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df1 = eta_df1.loc[eta_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df2 = eta_df2.loc[eta_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df3 = eta_df3.loc[eta_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df4 = eta_df4.loc[eta_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df5 = eta_df5.loc[eta_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]



In [138]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [118]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction RPC0 any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [119]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(6):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(6):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]
        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

In [31]:
show(combined_df[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [32]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Chi2 Analysis

In [149]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1.333

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], histtype='step', linewidth=1.3, log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas Off')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [177]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
df_chi2_unfiltered = [df_recon_no_rpc0_4, df_recon_no_rpc1_4, df_recon_no_rpc2_4, df_recon_no_rpc3_4, df_recon_no_rpc4_4, df_recon_no_rpc5_4]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(len(df_chi2)):
    df_chi2[i]['Chi_squared'] = df_chi2[i]['RSS'] / 1

plt.figure(figsize=(10, 6))
for i in range(len(df_chi2)):
    plt.hist(df_chi2[i]['Chi_squared'], bins=500, alpha=0.75, 
             label=label_rpc[i], log = True)
plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Atlas ON')
plt.ylabel('Number of Occurance')
plt.legend()
plt.grid(True)
plt.show()

In [155]:
print(df_recon_no_rpc0_4_filtered.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1132 non-null   int64  
 1   Event Number      1132 non-null   int64  
 2   Direction         1132 non-null   object 
 3   Slope             1132 non-null   float64
 4   Intercept         1132 non-null   float64
 5   Slope_error       1132 non-null   int64  
 6   Intercept_error   1132 non-null   int64  
 7   Used Coordinates  1132 non-null   object 
 8   RSS               1132 non-null   float64
 9   Chi_squared       1132 non-null   float64
 10  expected hits     1132 non-null   float64
dtypes: float64(5), int64(4), object(2)
memory usage: 97.4+ KB
None


In [12]:
df_chi2 = [df_recon_no_rpc0_4_filtered, df_recon_no_rpc1_4_filtered, df_recon_no_rpc2_4_filtered, df_recon_no_rpc3_4_filtered, df_recon_no_rpc4_4_filtered, df_recon_no_rpc5_4_filtered]
label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']
df_eta_list = []
df_phi_list = []
                    
Variance_eta = ( 0.5 * 3.09375) ** 2 / 12
Variance_phi = ( 0.5 * 2.8125) ** 2 / 12

for df in df_chi2:
    df_eta = df[df['Direction'] == 'eta']
    df_phi = df[df['Direction'] == 'phi']
    
    # Append to lists
    df_eta_list.append(df_eta)
    df_phi_list.append(df_phi)

# Calculate Chi_squared values
for df in df_eta_list:
    df['Chi_squared'] = df['RSS'] / Variance_eta 
for df in df_phi_list:
    df['Chi_squared'] = df['RSS'] / Variance_phi 

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_eta_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log = True)


plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Eta Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
for i, df in enumerate(df_phi_list):
    plt.hist(df['Chi_squared'], bins=500, alpha=0.75, label=label_rpc[i], log=True)

plt.title('Histogram of Chi squared values')
plt.xlabel('Chi squared value Phi Atlas Off')
plt.ylabel('Number of Occurrence')
plt.legend()
plt.grid(True)
plt.show()

C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_eta  # This is the same as just using RSS directly
C:\Users\Peter\AppData\Local\Temp\ipykernel_3680\220974511.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chi_squared'] = df['RSS'] / Variance_phi  # This is the same as just using RSS directly


In [187]:
show(df_eta_list[0])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [151]:
show(df_recon_no_rpc0_4_filtered)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [13]:
show(eta_df0)

NameError: name 'eta_df0' is not defined

In [34]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]


efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [37]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(3):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [38]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(3):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(3):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        total_valid = combined_df[i].shape[0]

        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(3):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 5 AtlasOFF')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

Separate them into TDCs Using a reverse Mapping

In [43]:
show(phi_df1)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [136]:
reversed_mapping = {
    ('rpc0', 'eta'): [(0, 32, 0)],
    ('rpc0', 'phi'): [(0, 64, 0)],
    ('rpc1', 'eta'): [(0, 32, 0)],
    ('rpc1', 'phi'): [(0, 64, 1)],
    ('rpc2', 'eta'): [(0, 32, 1)],
    ('rpc2', 'phi'): [(0, 64, 1)],
    ('rpc3', 'eta'): [(0, 32, 2)],
    ('rpc3', 'phi'): [(0, 64, 2)],
    ('rpc4', 'eta'): [(0, 32, 3)],
    ('rpc4', 'phi'): [(0, 64, 3)],
    ('rpc5', 'eta'): [(0, 32, 3)],
    ('rpc5', 'phi'): [(0, 64, 4)]
}

for key, value in reversed_mapping.items():
    print(key, ":", value)
    
def compute_tdc_number(rpc, direction, expected_hits, mapping):
    if (rpc, direction) in mapping:
        for (start, end, chip) in mapping[(rpc, direction)]:  

            if start <= expected_hits <= end:

                return int(chip)
    return None  

print(compute_tdc_number('rpc1', 'eta', 12.021, reversed_mapping))

('rpc0', 'eta') : [(0, 32, 0)]
('rpc0', 'phi') : [(0, 64, 0)]
('rpc1', 'eta') : [(0, 32, 0)]
('rpc1', 'phi') : [(0, 64, 1)]
('rpc2', 'eta') : [(0, 32, 1)]
('rpc2', 'phi') : [(0, 64, 1)]
('rpc3', 'eta') : [(0, 32, 2)]
('rpc3', 'phi') : [(0, 64, 2)]
('rpc4', 'eta') : [(0, 32, 3)]
('rpc4', 'phi') : [(0, 64, 3)]
('rpc5', 'eta') : [(0, 32, 3)]
('rpc5', 'phi') : [(0, 64, 4)]
0


In [64]:
show(eta_df0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [137]:
eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]
phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]
rpc_number = ['rpc0', 'rpc1', 'rpc2', 'rpc3', 'rpc4', 'rpc5']
for i in range(6):
    eta_df[i]['TDC Number'] = eta_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], -row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )
    phi_df[i]['TDC Number'] = phi_df[i].apply(
        lambda row: compute_tdc_number(rpc_number[i], row['Direction'], row['expected hits'], reversed_mapping)
        if pd.notna(row['expected hits']) else None,
        axis = 1
    )

    

In [138]:
combined_eta_df = pd.concat(eta_df, ignore_index=True)
combined_phi_df = pd.concat(phi_df, ignore_index=True)

# Now concatenate the two resulting DataFrames vertically as well
combined_df = pd.concat([combined_eta_df, combined_phi_df], ignore_index=True)

In [111]:
show(combined_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [112]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7542 entries, 0 to 7541
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Number    7542 non-null   int64  
 1   Direction       7542 non-null   object 
 2   expected hits   7542 non-null   float64
 3   rpc_number      5388 non-null   object 
 4   midpoint        5388 non-null   float64
 5   hit_difference  5388 non-null   float64
 6   TDC Number      7542 non-null   int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 412.6+ KB
None


In [139]:
tdc_numbers = combined_df['TDC Number'].unique()  

tdc_dfs = {}

for tdc_number in tdc_numbers:
    tdc_dfs[tdc_number] = combined_df[combined_df['TDC Number'] == tdc_number]

df_tdc_0 = tdc_dfs[0]

df_tdc_1 = tdc_dfs[1]
df_tdc_2 = tdc_dfs[2]
df_tdc_3 = tdc_dfs[3]
df_tdc_4 = tdc_dfs[4]

In [115]:
show(df_tdc_0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [140]:
thresholds = 3
df_tdc = [df_tdc_0, df_tdc_1, df_tdc_2, df_tdc_3, df_tdc_4]

efficiencies = []

label_tdc = ['tdc0', 'tdc1', 'tdc2', 'tdc3', 'tdc4']

for i in range(5):
    valid_hits = df_tdc[i].dropna(subset=['hit_difference'])
    count_valid = valid_hits[valid_hits['hit_difference'].abs() <= thresholds].shape[0]
    total_valid = df_tdc[i].shape[0]
    efficiency = count_valid / total_valid if total_valid > 0 else 0
    efficiencies.append(efficiency)

plt.figure(figsize=(10, 6))
plt.bar(label_tdc, efficiencies, color='blue')
plt.title('Efficiency vs TDC Number for ETA Direction any 5 AtlasON')
plt.xlabel('TDC Number')
plt.ylabel('Efficiency')
plt.ylim(0, 1)  # Set y-axis limits to show from 0 to 100% efficiency
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()